### Analysis 

In [ ]:
import numpy as np
import pylab as pl
import pandas as pd
# import matplotlib.pyplot as plt 
# %matplotlib inline
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import cross_val_score, GridSearchCV
import os
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV


from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB


# import seaborn as sn
import csv
from tqdm import tqdm_notebook as tqdm
import os
import seaborn as sns
import pandas as pd
from importlib import reload
import matplotlib.pyplot as plt

from collections import Counter
from nltk.stem.porter import *
from tqdm import tqdm
from wordcloud import WordCloud, STOPWORDS
from pprint import pprint
import tweepy
import random
import emoji
import json
import re
import subprocess
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

import random
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from dateutil.parser import parse
import requests
import plotly.graph_objects as go
import emoji
import demoji

In [ ]:
# !pip install demoji
demoji.download_codes()

In [ ]:
def removePattern(text, pattern):

    r = re.findall(pattern, text)

    for i in r:

        text = re.sub(i, '', text)

    return text

In [ ]:
def preprocessTweet(tweet_text, only_english=True, stemming=False, remove_more_than=3):

    """

    Function to perform various pre-processing steps on  

    tweet text



    Parameters

    ----------

    only_english : bool

        removes characters from other languages when set to True;

        defaults to False

    

    remove_more_than : int

        remove words more than three characters

    

    stemming : bool

        performs stemming if set to True; defaults to False

        

    tweet_text : str

        < 280 character string of text

    

    Returns

    -------

    tweet_text

        processed tweet

    """

    

    tweet_text = tweet_text.lower()

    tweet_text = removePattern(tweet_text, "@[\w]*") # remove twitter handles

    tweet_text = removePattern(tweet_text, "&[\w]*") # remove &amp

    tweet_text = re.sub('[!@$:).;/#,*$?।&…"]', '', tweet_text) # remove special characters, punctuations

    tweet_text = removePattern(tweet_text, "http[\w]*")

    if only_english:

        tweet_text = tweet_text.encode('ascii', 'ignore').decode('ascii') # remove emojis and other languages

    else:

        tweet_text = emoji.get_emoji_regexp().sub(u'', tweet_text) # just remove emojis

    

    if stemming:

        stemmer = PorterStemmer()

        tweet_text = ' '.join([stemmer.stem(word).strip() for word in tweet_text.split() \

                               if len(word)>=remove_more_than and word not in STOPWORDS]) # removing stems, short words and stop words

    else:

        tweet_text = ' '.join([word.strip() for word in tweet_text.split() \

                               if len(word)>=remove_more_than and word not in STOPWORDS]) # removing short words and stop words
    
    return tweet_text

### Get all features

In [ ]:
#extract average tweets per hour
"""['account_id', 'followers', 'friends', 'favourites', 'account_creation', 'is_verified', 'name', 'screen_name', 
'description', 'tweet_id', 'tweet_creation', 'tweet_text', 'is_RT']"""

def extract_profile_features(root_dir):
    features = {} 
    ctime = pd.Timestamp.now(tz='UTC')
    for subdir, dirs, files in os.walk(root_dir):
        for file in tqdm(files):
            filename = subdir + os.sep + file
            with open(filename,'r')as f:
                data = csv.reader(f)
                r = 0
                last_tweet = 0
                cur_hr = 1.0
                avg_list = [] 
                userid = 0
                words = ""
                usr_features = {}
                for row in data:
                    r+=1
                    if r == 1:
                        continue  
                    #first tweet (the latest)
                    try:
                        otime = pd.to_datetime(row[10])
                        cur_time = (ctime-otime).total_seconds()
                        if abs(cur_time-last_tweet) <= 3600:
                            cur_hr += 1.0
                        else:
                            last_tweet = cur_time
                            avg_list.append(cur_hr)
                            cur_hr = 1.0
                        if r == 2:
                            userid = row[0]
                            usr_features['followers'] = row[1]
                            usr_features['creation_date'] = row[4]
                            usr_features['friends'] = row[2]
                            usr_features['favourites'] = row[3]
                            usr_features['friend_to_follow'] = float(row[2])/(float(row[1])+1)
                            usr_features['name_len'] = len(row[6])
                            usr_features['bio_len'] = len(row[8])
                            usr_features['scr_len'] = len(row[7])
                            usr_features['scr_len_words'] = len(row[7].strip(' ').split(' '))
                            usr_features['bio_len_words'] = len(row[8].strip(' ').split(' '))
                            usr_features['tweet'] = preprocessTweet(row[-2])
                        if(len(row) == 13):
                            usr_features['tweet'] += preprocessTweet(row[-2])
                    except:
                        pass
#                         print(row)
#                         print(file)


                usr_features['no_of_tweets'] = r - 1
                if len(avg_list) == 0:
                    avg_list.append(0)
                usr_features['avg_tweets'] = sum(avg_list)/(len(avg_list))
#                 print(len(usr_features))
                features[userid] = usr_features
    return features

In [ ]:
deleted_account_feature = extract_profile_features("../deleted_accounts")
suspended_account_feature = extract_profile_features("../suspended_accounts")
normal_account_feature = extract_profile_features("../normal_accounts")

df1 = pd.DataFrame.from_dict(deleted_account_feature).T
df2 = pd.DataFrame.from_dict(suspended_account_feature).T
df3 = pd.DataFrame.from_dict(normal_account_feature).T

df1 = df1.dropna()
df2 = df2.dropna()
df3 = df3.dropna()

In [ ]:
df1.to_csv("deleted_account_features.csv")
df2.to_csv("suspended_account_features.csv")
df3.to_csv("normal_account_features.csv")

In [ ]:
df1 = pd.read_csv("deleted_account_features.csv")
df2 = pd.read_csv("suspended_account_features.csv")
df3 = pd.read_csv("normal_account_features.csv")

In [ ]:
df1.head()

In [ ]:
df1['creation_date'] = pd.to_datetime(df1['creation_date'])
df2['creation_date'] = pd.to_datetime(df2['creation_date'])
df3['creation_date'] = pd.to_datetime(df3['creation_date'])

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes
from mpl_toolkits.axes_grid.inset_locator import inset_axes

def plot_creation_date(dt_col,label,x1=1,x2=1,y1=1,y2=1):
    ser = pd.Series(1, index= dt_col).resample('M', how='count')
    ser.index = ser.index.to_period('M')
    tot_counts = ser.sum()
    ser /= tot_counts
#     print(ser)
    ax = ser.plot()
    ax.set(xlabel="Creation_Date", ylabel="Percentage of Accouts Created")
    ax.set_title(label)
    axins = inset_axes(ax, 2,1.5 , loc=2,bbox_to_anchor= (0.2, 0.85),bbox_transform= ax.figure.transFigure) # no zoom
#     axins = zoomed_inset_axes(ax, 2.0, loc=2) # zoom-factor: 2.5, location: upper-left
    axins = ser.plot(marker='o')
    axins.set(xlabel="")
#    x1, x2, y1, y2 = 2018, 60, 3.7, 4.6 # specify the limits
    axins.set_xlim(x1, x2) # apply the x-limits
#     axins.set_ylim(y1, y2) # apply the y-limits

In [ ]:
plot_creation_date(df1['creation_date'], "deleted_accounts", x1 = '2019-02',x2 = '2019-06', y1 = 0.06, y2 = 0.12)

In [ ]:
plot_creation_date(df2['creation_date'], "suspended_accounts", x1= '2019-02', x2= '2019-06', y1= 0.05, y2= 0.30)

In [ ]:
plot_creation_date(df3['creation_date'],"normal_accounts",x1 = '2019-02',x2 = '2019-06',y1 = 0.004,y2 = 0.06)

### Account behaviour before/after suspension/deletion.

### Generating Dataset
Identifying a set of tweets for each account before and after suspension date. 

### Step-Wise method for the same :-
-  Convert userid to username 
-  Query username between start-date (29th April - 2 months below suspension date)  and end-date (8th December - 2 months after suspension was lifted).
-  Store json files for each user as userid_old.json and userid_new.json 

In [ ]:
api_details = {
"consumer_key" : "",
"consumer_secret" : "",
"access_token" : "",
"access_token_secret" : "",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys = []
keys.append(api_details)

api_details = {
"consumer_key" : "",
"consumer_secret" : "",
"access_token" : "",
"access_token_secret" : "",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys.append(api_details)

api_details = {
"consumer_key" : "",
"consumer_secret" : "",
"access_token" : "",
"access_token_secret" : "",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys.append(api_details)

api_details = {
"consumer_key" : "",
"consumer_secret" : "",
"access_token" : "",
"access_token_secret" : "",
"in_use" : False,
"rate_limit_hit" : False,
"error_status" : False
}

keys.append(api_details)

def doauth(index):
    auth = tweepy.OAuthHandler(keys[index]['consumer_key'], keys[index]['consumer_secret'])
    auth.set_access_token(keys[index]['access_token'], keys[index]['access_token_secret'])
    api = tweepy.API(auth)
    return api

idx = 0    

In [ ]:
def userid_to_username(userid):
    try:
        api = doauth(0)
        user = api.get_user(userid)
        return user.screen_name
    except:
        return -1

In [ ]:
def query_tweets(uid,screen_name,begin_date,end_date,folder,file):
    
    params = "twitterscraper 'from:" + screen_name + "' -bd " + begin_date + " -ed " + end_date + " -p 15" + " -o " + "../personality_analysis/" + str(folder) + "/" + str(uid) + file + ".json"
    print(params)
    try:
        op = subprocess.run(params,shell=True)
    except:
        print("QUERY_FAIL","WGGGY")
        pass

In [ ]:
import os.path

def extract_accounts(root_dir):
    old_tweets_begin = "2019-04-29"
    old_tweets_end = "2019-06-29"
    new_tweets_begin = "2019-10-08"
    new_tweets_end = "2019-12-08"
    ftype = "undeleted_accounts"
    
    if("unsuspended" in root_dir):
        ftype = "unsuspended_accounts"
        
    for subdir, dirs, files in os.walk(root_dir):
        for file in tqdm(files):
            uid = file.split(".")[0]
            screen_name = userid_to_username(uid)
            if(screen_name == -1):
                print("WRONG")
                continue
                
            fpath1 = "../personality_analysis/" + str(ftype) + "/" + str(uid) +  "_old.json"
            fpath2 = "../personality_analysis/" + str(ftype) + "/" + str(uid) +  "_new.json"
            
            if (os.path.isfile(fpath1)):
                continue
            if (os.path.isfile(fpath2)):
                continue
                
            
            
            try:
                query_tweets(uid,screen_name,old_tweets_begin,old_tweets_end,ftype,"_old")
            except:
                print("OLD_QUERY_FAIL",uid)
                pass
            
            try:
                query_tweets(uid,screen_name,new_tweets_begin,new_tweets_end,ftype,"_new")
            except:
                print("NEW_QUERY_FAIL",uid)
                pass
            
            try:
                fpath = "../personality_analysis/" + str(ftype) + "/" + str(uid) +  "_old.json"
                df = pd.read_json(fpath)
                df.drop(['screen_name','text_html','timestamp_epochs','tweet_url','user_id','username'], axis=1, inplace=True)
                df.to_json(fpath)
            except:
                print("OLD_WRITE_FAIL",uid)
                pass
            
            try:
                fpath = "../personality_analysis/" + str(ftype) + "/" + str(uid) +  "_new.json"
                df = pd.read_json(fpath)
                df.drop(['screen_name','text_html','timestamp_epochs','tweet_url','user_id','username'], axis=1, inplace=True)
                df.to_json(fpath)
            except:
                print("NEW_WRITE_FAIL",uid)
                pass

In [ ]:
extract_accounts("../undeleted_accounts/")

In [ ]:
extract_accounts("../unsuspended_accounts/")

In [ ]:
"""['account_id', 'followers', 'friends', 'favourites', 'account_creation', 'is_verified', 'name', 'screen_name', 
'description', 'tweet_id', 'tweet_creation', 'tweet_text', 'is_RT']"""

compare_date = parse("2019-10-09 00:00:00+00:00")
def seperate_tweets_accounts(root_dir,target_dir):
    cnt = 0
    for subdir, dirs, files in os.walk(root_dir):
        for file in tqdm(files):
#             print(file)
            if(file != '18732540.csv'):
                continue
            
            older_tweets = []
            newer_tweets = []
            filename = subdir + os.sep + file
            
            with open(filename,'r')as f:
                data = csv.reader(f)
                r = 0
                for row in data:
                    r+=1
                    if r == 1:
                        older_tweets.append(row)
                        newer_tweets.append(row)
                        continue  
                    try:
                        otime = parse(row[10])
                        if(otime < compare_date):
                            print(otime,row[-4])
                            older_tweets.append(row)
                        else:
                            newer_tweets.append(row)
                    except:
                        pass
            
            limit = 100
#             if ( len(older_tweets) >= limit and len(newer_tweets) >= limit ) :
#                 fname1 = target_dir+file.split(".")[0]+ "_old.csv"
#                 fname2 = target_dir+file.split(".")[0]+ "_new.csv"

#                 with open(fname1,'a+') as ff:
#                     for ind,val in enumerate(older_tweets):
#                         if(ind == 0):
#                             continue
#                         ff.write(older_tweets[ind][-2] + '\n')
                        
#                 with open(fname2,'a+') as ff:
#                     for ind,val in enumerate(newer_tweets):
#                         if(ind == 0):
#                             continue
#                         ff.write(newer_tweets[ind][-2] + '\n')

In [ ]:
seperate_tweets_accounts('../unsuspended_accounts/' , '../personality_analysis/unsuspended_accounts/')

In [ ]:
seperate_tweets_accounts('../undeleted_accounts/' , '../personality_analysis/undeleted_accounts/')

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return False

In [ ]:
def linguistic_analysis(root_dir):
    for subdir, dirs, files in os.walk(root_dir):
        for file in tqdm(files):
            older_tweets = []
            newer_tweets = []
            filename = subdir + os.sep + file
            
            percentage_non_english = []
            if(file != '1118084130149928960.csv'):
                cont
            with open(filename,'r') as f:
                data = csv.reader(f)
                total_count = 0
                r = 0
                for row in data:
                    r+=1
                    if r == 1:
                        continue  
                    try:
                        tweet_text = demoji.replace(row[-2])
                        print(tweet_text)
                        if(isEnglish(tweet_text) == False):
                            print(tweet_text,isEnglish(mytweet),"FFFFFF")
                            total_count += 1
                    except:
                        pass
                
                perc = (total_count/r)*100
                percentage_non_english.append(total_count/r * 100)
                
            
            break

In [ ]:
linguistic_analysis("../deleted_accounts.csv")

In [ ]:
# print("F")

### Querying IBM Personality.

In [ ]:
def create_result(source_file,target_file,url,apikey):
    headers = {
        'Content-Type': 'text/plain;charset=utf-8',
        'Accept': 'application/json',
    }

    params = (
        ('version', '2017-10-13'),
    )

    data = open(source_file,'rb').read()
    
    response = requests.post( url + '/v3/profile?version=2017-10-13&consumption_preferences=true&raw_scores=true', headers=headers, data=data, auth=('apikey', apikey))
    dt = response.json()

    with open(target_file, 'w') as fp:
        json.dump(dt, fp)    

In [ ]:
# root_dir = '/Deleted_Accounts/personality_analysis/unsuspended_accounts'
# target_dir = '/Deleted_Accounts/personality_analysis/unsuspended_accounts_scores'
# api_key = ''
# url_val = 'https://api.us-south.personality-insights.watson.cloud.ibm.com/...'

root_dir = '/Deleted_Accounts/personality_analysis/undeleted_accounts'
target_dir = '/Deleted_Accounts/personality_analysis/undeleted_accounts_scores'



api_key = ''
url_val = # 'https://api.eu-gb.personality-insights.watson.cloud.ibm.com/...'

for subdir, dirs, files in os.walk(root_dir):
    for file in tqdm(files):
        filename = subdir + os.sep + file
        create_result(filename , target_dir + '/' + file.split('.')[0] + '.json',url_val,api_key)

In [ ]:
# Dictionary of people with following schema
#  { userid : { 'old_vals' : { 'name_property1' : {child_name : value ,..... } ,  } } , new_vals ...same }
## List of 25 properties
name_properites = []
def get_properties(path):
    data = {}
    with open(path,'r') as f:
        data = json.load(f)
        for keys in data['personality']:
            mtype = (keys['name'])
            print(mtype)
            for each in keys['children']:
                name_properites.append(each['name'])

In [ ]:
user_scores = {};

def load_vals(path):
    data = {}
    with open(path,'r') as f:
        data = json.load(f)
    
    
    uid = path.split('/')[-1].split('_')[0]
    print(path)
    if uid not in user_scores:
        user_scores[uid] = {'deleted':{'old_vals':[],'new_vals':[]},'suspended':{'old_vals':[],'new_vals':[] } }  
    
    try:   
        if('old' in path and 'undeleted' in path):
            for keys in data['personality']:
                mtype = (keys['name'])
                for each in keys['children']:
                    user_scores[uid]['deleted']['old_vals'].append(each['percentile'])

        if('old' in path and 'unsuspended' in path):
            for keys in data['personality']:
                mtype = (keys['name'])
                for each in keys['children']:
                    user_scores[uid]['suspended']['old_vals'].append(each['percentile'])

        if('new' in path and 'undeleted' in path):
            for keys in data['personality']:
                mtype = (keys['name'])
                for each in keys['children']:
                    user_scores[uid]['deleted']['new_vals'].append(each['percentile'])

        if('new' in path and 'unsuspended' in path):
            for keys in data['personality']:
                mtype = (keys['name'])
                for each in keys['children']:
                    user_scores[uid]['suspended']['new_vals'].append(each['percentile'])
    except:
        pass
                
#                 user_scores[uid]['deleted']['old_vals'].append()
#                 name_properites.append(each['name'])
#     print(user_scores[uid])

In [ ]:
# root_dir = '/Deleted_Accounts/personality_analysis/undeleted_accounts_scores'

# for subdir, dirs, files in os.walk(root_dir):
#     for file in tqdm(files):
#         filename = subdir + os.sep + file
#         load_vals(filename)

In [ ]:
root_dir = '/Deleted_Accounts/personality_analysis/unsuspended_accounts_scores'
for subdir, dirs, files in os.walk(root_dir):
    for file in tqdm(files):
        filename = subdir + os.sep + file
        load_vals(filename)
#         break

In [ ]:
key_to_be_removed = []
for key,value in user_scores.items():
    if ( (value['suspended']['old_vals'] == []) or  (value['suspended']['new_vals'] == []) ):
        key_to_be_removed.append(key)

In [ ]:
for each in key_to_be_removed:
    del(user_scores[each])

In [ ]:
root_dir = '/Deleted_Accounts/personality_analysis/unsuspended_accounts_scores'
for subdir, dirs, files in os.walk(root_dir):
    for file in tqdm(files):
        filename = subdir + os.sep + file
        get_properties(filename)
        break

In [ ]:
def finding_mean_scores(user_scores):
    scoring_list_old = []
    scoring_list_new = []
    
    counter = 0
    for key,value in user_scores.items():
        
        counter+=1
        p_old = (value['suspended']['old_vals'])
        p_new = (value['suspended']['new_vals'])
        for ind,val in enumerate(p_old):
            if scoring_list_old == []:
                scoring_list_old = p_old
            else:
                scoring_list_old[ind] += val
        
        for ind,val in enumerate(p_new):
            if scoring_list_new == []:
                scoring_list_new = p_new
            else:
                scoring_list_new[ind] += val
    
    scoring_list_old[:] = [x / counter for x in scoring_list_old]
    scoring_list_new[:] = [x / counter for x in scoring_list_new]
    
    return  scoring_list_old,scoring_list_new
    

In [ ]:
scoring_list_old,scoring_list_new = finding_mean_scores(user_scores)

In [ ]:
def radial_plot(categories,old_vals,new_vals):
    
    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
          r=old_vals,
          theta=categories,
          fill='toself',
          name='Before Suspension'
    ))
    fig.add_trace(go.Scatterpolar(
          r=new_vals,
          theta=categories,
          fill='toself',
          name='After Suspension'
    ))

    fig.update_layout(
      polar=dict(
        radialaxis=dict(
          visible=True,
          range=[0, 0.8]
        )),
      showlegend=True
    )
    fig.show()

In [ ]:
radial_plot(name_properites[0:5],scoring_list_old[0:5],scoring_list_new[0:5])